# __Drug Pricing Prediction Model__

In this notebook, we'll aggregate and clean the data for the following notebooks, with the intent to create predictions of pharmaceutical drug prices.
* __national_average_drug_acquisition_cost.csv__: a dataset of (updated monthly)
    * A data dicationary can be found [here]('https://www.medicaid.gov/medicaid-chip-program-information/by-topics/prescription-drugs/ful-nadac-downloads/nadacdatadefinitions.pdf')
    * [Source data]('https://healthdata.gov/dataset/nadac-national-average-drug-acquisition-cost')

The following three files come are gathered from the Orange Book (the FDA's dataset on drug approvals).  Source data & the accompanying dictionary can be found [here]('https://www.fda.gov/drugs/drug-approvals-and-databases/orange-book-data-files').
* __products.txt__: specific information regarding products registered with the FDA
    * Trade name
    * Applicant
    * New Drug Application (NDA) Number
    * Product Number
    * __Approval Date__
    * Type
* __patent.txt__: patent data as available for each drug ([note]('https://www.fda.gov/drugs/development-approval-process-drugs/frequently-asked-questions-patents-and-exclusivity#What_is_the_difference_between_patents_a'), this is different from exclusivity).  Columns of interest:
    * New Drug Application (NDA) Number
    * Product Number
    * Patent Number
    * __Patent Expire Date__
* __exclusivity.txt__: data particular to the exclusive marketing rights granted by the FDA to the drug company for a particular drug.  Columns of interest:
    * New Drug Application (NDA) Number
    * Product Number
    * Exclusivity Code
    * __Exclusivity Date__

Those in bold are of peak interest.  Although the identified columns are those of clear interest, I'll leave the remaining columns in the datasets as I continue exploring.  

I'll use the New Drug Application (NDA) Number to join the information from these three files and then begin cleaning and exploring the data.

As you'll learn later, I'll combine the prices and patent/products/exclusivity datasets via a fuzzy string matching function.

Data in the following three panels can be used to access the API for some datasets used in this and other notebooks for this project.

In [30]:
# Data.wa.gov API info (Socrata?): 
    # key ID: *saved elsewhere*
    # key secret: *saved elsewhere*
    
    # app_key: *saved elsewhere*
    # secret token: *saved elsewhere*

In [31]:
# from sodapy import Socrata
# url = 'https://data.medicaid.gov'
# client = Socrata(url,'*saved elsewhere*', 
#                  username='*saved elsewhere*', 
#                  password='*saved elsewhere*')
# results = client.get('rk4x-vkz6.csv')
# results_df = pd.DataFrame.from_records(results)
# results_df.head(10)

In [32]:
# Create list of years for state drug utilization data
# years = list(range(1991,2019))

# Access the API and pull data
# url = 'https://data.medicaid.gov/resource/rk4x-vkz6.csv'
# response = requests.get(url)

In [33]:
import pandas as pd
import datetime as dt
import re
import gc
from fuzzywuzzy import fuzz

In [34]:
# Read in relevant files (the first 3 belong in a set and will be combined shortly)
drugs = pd.read_csv('products.txt', sep='~', engine='python')
patents = pd.read_csv('patent.txt', sep='~')
exclusivity = pd.read_csv('exclusivity.txt', sep='~')

prices = pd.read_csv('national_average_drug_acquisition_cost.csv')

In [35]:
drugs.describe(include='all')

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
count,37398,37398,37398,37398,37332,37398,37398.000000,37398.000000,17964,37398,37398,37398,37398,37398
unique,2518,281,6889,1643,3816,2,NaN,NaN,25,6548,2,2,3,1676
top,ACETAMINOPHEN; HYDROCODONE BITARTRATE,TABLET;ORAL,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,MYLAN,10MG,A,NaN,NaN,AB,"Approved Prior to Jan 1, 1982",No,No,RX,MYLAN PHARMACEUTICALS INC
freq,240,15630,210,1109,1305,27685,NaN,NaN,12294,5981,31412,33516,20693,1363
mean,NaN,NaN,NaN,NaN,NaN,NaN,94737.871063,1.948714,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,66672.698859,1.958603,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,50778.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,77296.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,91368.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
drugs.head()

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,VALEANT PHARMS INTL,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",Yes,Yes,RX,VALEANT PHARMACEUTICALS INTERNATIONAL
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,"Nov 26, 2012",No,No,RX,PERRIGO UK FINCO LTD PARTNERSHIP
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,"May 24, 2017",No,No,RX,RICONPHARMA LLC
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO PHARM,0.12%,A,208204,1,AB,"May 24, 2017",No,No,RX,TARO PHARMACEUTICAL INDUSTRIES LTD
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM;TOPICAL",CLINDAMYCIN PHOSPHATE,PERRIGO UK FINCO,1%,A,90785,1,AT,"Mar 31, 2010",No,No,RX,PERRIGO UK FINCO LTD PARTNERSHIP


In [37]:
drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37398 entries, 0 to 37397
Data columns (total 14 columns):
Ingredient             37398 non-null object
DF;Route               37398 non-null object
Trade_Name             37398 non-null object
Applicant              37398 non-null object
Strength               37332 non-null object
Appl_Type              37398 non-null object
Appl_No                37398 non-null int64
Product_No             37398 non-null int64
TE_Code                17964 non-null object
Approval_Date          37398 non-null object
RLD                    37398 non-null object
RS                     37398 non-null object
Type                   37398 non-null object
Applicant_Full_Name    37398 non-null object
dtypes: int64(2), object(12)
memory usage: 4.0+ MB


This appears to be an almost entirely complete dataset (with the exception of the 'Strength' and 'TE_Code' columns)!  Let's take a look at the number of unique values in each column (including the NaNs). 

In [38]:
# Swap out column name to determine number of unique values per column
drugs['Appl_No'].value_counts(dropna=False)

19630     53
19631     16
4056      15
21703     15
204824    14
62814     14
20280     13
20533     13
21342     12
20734     12
11719     12
206977    12
21137     12
21292     12
552       12
21924     12
18268     12
76187     12
21402     12
209713    12
20616     12
9218      12
210831    12
21210     12
21301     12
20678     12
21116     12
6488      11
17027     11
9149      11
          ..
86169      1
88216      1
209045     1
80020      1
86146      1
18554      1
88184      1
77941      1
84557      1
87294      1
18522      1
20569      1
9000       1
84026      1
88120      1
75830      1
84329      1
71728      1
90143      1
18458      1
88088      1
204823     1
77845      1
72162      1
61453      1
84674      1
86610      1
6441       1
200691     1
71683      1
Name: Appl_No, Length: 21940, dtype: int64

In [39]:
# Interesting that there are 53 drugs with the same application number (Appl_No).  What are they? They do have different strengths, so we'll leave them alone as they're not true duplicates
count = drugs[drugs['Appl_No']==19630]
count.head(10)

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
12164,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;200MG/100ML,N,19630,31,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12165,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;450MG/100ML,N,19630,37,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12166,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;900MG/100ML,N,19630,43,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12167,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;110MG/100ML,N,19630,1,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12168,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;200MG/100ML,N,19630,7,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12169,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;330MG/100ML,N,19630,13,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12170,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;450MG/100ML,N,19630,19,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12171,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;900MG/100ML,N,19630,25,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12174,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.075% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;75MG/100ML;200MG/100ML,N,19630,32,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12175,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.075% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;75MG/100ML;450MG/100ML,N,19630,38,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC


In [40]:
exclusivity.head()

,Appl_Type,Appl_No,Product_No,Exclusivity_Code,Exclusivity_Date
0,N,9190,1,ODE-64,"Apr 4, 2021"
1,N,11366,2,ODE-96,"Aug 7, 2022"
2,N,17858,1,ODE-29,"Aug 13, 2019"
3,N,20287,6,NPP,"May 16, 2022"
4,N,20287,5,NPP,"May 16, 2022"


In [41]:
exclusivity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779 entries, 0 to 1778
Data columns (total 5 columns):
Appl_Type           1779 non-null object
Appl_No             1779 non-null int64
Product_No          1779 non-null int64
Exclusivity_Code    1779 non-null object
Exclusivity_Date    1779 non-null object
dtypes: int64(2), object(3)
memory usage: 69.6+ KB


In [42]:
patents.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date
0,N,20571,1,6403569,"Apr 28, 2020",NaN,NaN,U-449,NaN,NaN
1,N,20571,1,6794370,"May 1, 2020",NaN,NaN,U-606,NaN,NaN
2,N,20571,2,6403569,"Apr 28, 2020",NaN,NaN,U-449,NaN,NaN
3,N,20571,2,6794370,"May 1, 2020",NaN,NaN,U-606,NaN,NaN
4,N,20610,1,7452872,"Aug 24, 2026",NaN,NaN,U-141,NaN,NaN


In [43]:
patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14348 entries, 0 to 14347
Data columns (total 10 columns):
Appl_Type                  14348 non-null object
Appl_No                    14348 non-null int64
Product_No                 14348 non-null int64
Patent_No                  14348 non-null object
Patent_Expire_Date_Text    14348 non-null object
Drug_Substance_Flag        2489 non-null object
Drug_Product_Flag          7804 non-null object
Patent_Use_Code            7753 non-null object
Delist_Flag                73 non-null object
Submission_Date            10085 non-null object
dtypes: int64(2), object(8)
memory usage: 1.1+ MB


Each drug application receives a number ('Appl_No').   An application may have many drugs on it, so each of these drugs on an application receive a product number ('Product_No').  Therefore, I can match up the drugs in the exculusitity and patent data files with those in the drugs data file by first comparing application numbers in each, and the secondarily confirming their product numbers.

In [44]:
# exclusivity_indexed=exclusivity.set_index(['Appl_No', 'Product_No'])
# exclusivity_indexed

In [45]:
# Merged with 'left' because there were some nan patent data that was unneeded
all_data = pd.merge(drugs, patents, on = ['Appl_No', 'Product_No'], how = 'left')
all_data.head()

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type_x,Appl_No,Product_No,TE_Code,Approval_Date,...,Type,Applicant_Full_Name,Appl_Type_y,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,VALEANT PHARMS INTL,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",...,RX,VALEANT PHARMACEUTICALS INTERNATIONAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,"Nov 26, 2012",...,RX,PERRIGO UK FINCO LTD PARTNERSHIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,"May 24, 2017",...,RX,RICONPHARMA LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO PHARM,0.12%,A,208204,1,AB,"May 24, 2017",...,RX,TARO PHARMACEUTICAL INDUSTRIES LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM;TOPICAL",CLINDAMYCIN PHOSPHATE,PERRIGO UK FINCO,1%,A,90785,1,AT,"Mar 31, 2010",...,RX,PERRIGO UK FINCO LTD PARTNERSHIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## __Update on Patent Data!__

Because I only have 1779 entries for patent dates, I figure I'll need more data for my predictive model.  As it turns out, [patents issued after 1995]('http://www.drugsdb.com/blog/how-long-is-a-drug-patent-good-for.html') are valid for 20 years from the patent application filing (assuming maintenance fees are paid every 3.5, 7.5, and 11.5 years after the patent is granted).  From this, I can extrapolate two things: 

* I should be able to populate the 'Patent_Expire_Date_Text' column based on 'Patent_Submission_Date'.  I'll add this information to a third column instead of filling in the NaN values of the 'Patent_Submission_Date' so I can identify any deviations  the results with the actual 'Patent_Expire_Date_Text' information that I imported earlier.
    * Additional factors to consider: 
        * Hatch-Waxman extension: A drug can obtain a patent extension of 5 years to make up the length of the FDA approval process.
        * Pediatric exclusivity extension: drugs tested on children can gain an extra 6 months of patent protection (this can be used twice)
        * Drug reformulations: i.e. turning an drug taken by injection into a nasal spray version, or modifying dosages, can extend a patent for an additional up to 5 years
        * New uses: Drugs whose new uses are discovered can obtain another 3 years of patent protection
        * Orphan drugs (those treating rare diseases) gain an additional 7 years of patent protection (and the FDA can't approve any competing generics during the time)
        * 30-Month Stays: Generics often issue a competing patent, and are sued by the brand-name company.  This initiates a 30-month stay on the FDA approval of the generic.
            * Few drug companies can take advantage of this
        * Most of the methods above can be combined to secure a longer patent
* Any drugs with patent issue dates before 1995 may not be valid for prediction as the law governing these patents apparently changed

I'll evaluate the dates we currently have to see if a pattern is evident, before combining them with new estimations based on the factors above.

In [46]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49496 entries, 0 to 49495
Data columns (total 22 columns):
Ingredient                 49496 non-null object
DF;Route                   49496 non-null object
Trade_Name                 49496 non-null object
Applicant                  49496 non-null object
Strength                   49429 non-null object
Appl_Type_x                49496 non-null object
Appl_No                    49496 non-null int64
Product_No                 49496 non-null int64
TE_Code                    19892 non-null object
Approval_Date              49496 non-null object
RLD                        49496 non-null object
RS                         49496 non-null object
Type                       49496 non-null object
Applicant_Full_Name        49496 non-null object
Appl_Type_y                14348 non-null object
Patent_No                  14348 non-null object
Patent_Expire_Date_Text    14348 non-null object
Drug_Substance_Flag        2489 non-null object
Drug_Product

In [47]:
# Merge in the data from exclusivity.txt
all_data = pd.merge(all_data, exclusivity, on=['Appl_No', 'Product_No'], how='left')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58567 entries, 0 to 58566
Data columns (total 25 columns):
Ingredient                 58567 non-null object
DF;Route                   58567 non-null object
Trade_Name                 58567 non-null object
Applicant                  58567 non-null object
Strength                   58500 non-null object
Appl_Type_x                58567 non-null object
Appl_No                    58567 non-null int64
Product_No                 58567 non-null int64
TE_Code                    20466 non-null object
Approval_Date              58567 non-null object
RLD                        58567 non-null object
RS                         58567 non-null object
Type                       58567 non-null object
Applicant_Full_Name        58567 non-null object
Appl_Type_y                23361 non-null object
Patent_No                  23361 non-null object
Patent_Expire_Date_Text    23361 non-null object
Drug_Substance_Flag        4875 non-null object
Drug_Product

In [48]:
all_data.head()

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type_x,Appl_No,Product_No,TE_Code,Approval_Date,...,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date,Appl_Type,Exclusivity_Code,Exclusivity_Date
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,VALEANT PHARMS INTL,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,"Nov 26, 2012",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,"May 24, 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO PHARM,0.12%,A,208204,1,AB,"May 24, 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM;TOPICAL",CLINDAMYCIN PHOSPHATE,PERRIGO UK FINCO,1%,A,90785,1,AT,"Mar 31, 2010",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Comparing this and the next cell - seeing if there's a way to merge by name?
prices['NDC Description'].value_counts()

METFORMIN HCL 500 MG TABLET                22455
METFORMIN HCL 1,000 MG TABLET              18756
LISINOPRIL 20 MG TABLET                    17532
AMLODIPINE BESYLATE 5 MG TAB               17420
MONTELUKAST SOD 10 MG TABLET               17061
METFORMIN HCL 850 MG TABLET                16958
AMLODIPINE BESYLATE 10 MG TAB              16789
LISINOPRIL 10 MG TABLET                    16727
LOSARTAN POTASSIUM 50 MG TAB               16238
LISINOPRIL 40 MG TABLET                    15932
LOSARTAN POTASSIUM 100 MG TAB              15730
LISINOPRIL 5 MG TABLET                     15105
GABAPENTIN 300 MG CAPSULE                  14016
DONEPEZIL HCL 5 MG TABLET                  13696
IBUPROFEN 200 MG TABLET                    13575
AMLODIPINE BESYLATE 2.5 MG TAB             13157
FINASTERIDE 5 MG TABLET                    12918
DONEPEZIL HCL 10 MG TABLET                 12793
CITALOPRAM HBR 40 MG TABLET                12645
CLOPIDOGREL 75 MG TABLET                   12539
OMEPRAZOLE DR 20 MG 

In [50]:
# all_data.head()
all_data['DF;Route'].value_counts()

TABLET;ORAL                                                                      23776
CAPSULE;ORAL                                                                      7617
INJECTABLE;INJECTION                                                              6774
TABLET, EXTENDED RELEASE;ORAL                                                     3179
CAPSULE, EXTENDED RELEASE;ORAL                                                    1747
SOLUTION;ORAL                                                                      773
SOLUTION;SUBCUTANEOUS                                                              659
SOLUTION;INTRAVENOUS                                                               594
SOLUTION/DROPS;OPHTHALMIC                                                          580
CREAM;TOPICAL                                                                      567
INJECTABLE;INTRAVENOUS                                                             558
CAPSULE, DELAYED RELEASE;ORAL              

In [51]:
# Separate the 'DF' (drug format) and 'Route' information into different columns
all_data['dosage_form'] = all_data['DF;Route'].str.split(';', n=1).str[0]
all_data['route'] = all_data['DF;Route'].str.split(';').str[-1]
all_data.head()

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type_x,Appl_No,Product_No,TE_Code,Approval_Date,...,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date,Appl_Type,Exclusivity_Code,Exclusivity_Date,dosage_form,route
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,VALEANT PHARMS INTL,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AEROSOL, FOAM",RECTAL
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,"Nov 26, 2012",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AEROSOL, FOAM",TOPICAL
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,"May 24, 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AEROSOL, FOAM",TOPICAL
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO PHARM,0.12%,A,208204,1,AB,"May 24, 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AEROSOL, FOAM",TOPICAL
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM;TOPICAL",CLINDAMYCIN PHOSPHATE,PERRIGO UK FINCO,1%,A,90785,1,AT,"Mar 31, 2010",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AEROSOL, FOAM",TOPICAL


In [52]:
# Create a list of column names for reordering columns in next cell
cols = list(all_data.columns.values)
cols

['Ingredient',
 'DF;Route',
 'Trade_Name',
 'Applicant',
 'Strength',
 'Appl_Type_x',
 'Appl_No',
 'Product_No',
 'TE_Code',
 'Approval_Date',
 'RLD',
 'RS',
 'Type',
 'Applicant_Full_Name',
 'Appl_Type_y',
 'Patent_No',
 'Patent_Expire_Date_Text',
 'Drug_Substance_Flag',
 'Drug_Product_Flag',
 'Patent_Use_Code',
 'Delist_Flag',
 'Submission_Date',
 'Appl_Type',
 'Exclusivity_Code',
 'Exclusivity_Date',
 'dosage_form',
 'route']

In [53]:
# Reorder column names with the new 'df' and 'route' columns in a more appropriate place, and drop the original 'DF;Route' column
cols = ['Ingredient', 'dosage_form', 'route', 'Trade_Name', 'Strength', 'Applicant', 'Appl_Type_x', 'Appl_No', 'Product_No', 'TE_Code', 'Approval_Date', 'RLD', 'RS', 'Type',
 'Applicant_Full_Name', 'Appl_Type_y', 'Patent_No', 'Patent_Expire_Date_Text', 'Drug_Substance_Flag', 'Drug_Product_Flag', 'Patent_Use_Code', 'Delist_Flag',
 'Submission_Date', 'Appl_Type', 'Exclusivity_Code', 'Exclusivity_Date']
all_data = all_data[cols]
all_data.head()

,Ingredient,dosage_form,route,Trade_Name,Strength,Applicant,Appl_Type_x,Appl_No,Product_No,TE_Code,...,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date,Appl_Type,Exclusivity_Code,Exclusivity_Date
0,BUDESONIDE,"AEROSOL, FOAM",RECTAL,UCERIS,2MG/ACTUATION,VALEANT PHARMS INTL,N,205613,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,PERRIGO UK FINCO,A,78337,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,RICONPHARMA LLC,A,207144,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,TARO PHARM,A,208204,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM",TOPICAL,CLINDAMYCIN PHOSPHATE,1%,PERRIGO UK FINCO,A,90785,1,AT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58567 entries, 0 to 58566
Data columns (total 26 columns):
Ingredient                 58567 non-null object
dosage_form                58567 non-null object
route                      58567 non-null object
Trade_Name                 58567 non-null object
Strength                   58500 non-null object
Applicant                  58567 non-null object
Appl_Type_x                58567 non-null object
Appl_No                    58567 non-null int64
Product_No                 58567 non-null int64
TE_Code                    20466 non-null object
Approval_Date              58567 non-null object
RLD                        58567 non-null object
RS                         58567 non-null object
Type                       58567 non-null object
Applicant_Full_Name        58567 non-null object
Appl_Type_y                23361 non-null object
Patent_No                  23361 non-null object
Patent_Expire_Date_Text    23361 non-null object
Drug_Substa

In [55]:
all_data.columns = map(str.lower, all_data.columns)
all_data.head()

,ingredient,dosage_form,route,trade_name,strength,applicant,appl_type_x,appl_no,product_no,te_code,...,patent_no,patent_expire_date_text,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date
0,BUDESONIDE,"AEROSOL, FOAM",RECTAL,UCERIS,2MG/ACTUATION,VALEANT PHARMS INTL,N,205613,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,PERRIGO UK FINCO,A,78337,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,RICONPHARMA LLC,A,207144,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,TARO PHARM,A,208204,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM",TOPICAL,CLINDAMYCIN PHOSPHATE,1%,PERRIGO UK FINCO,A,90785,1,AT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## __Name Concatenation__
In the next cell, I'm combining the values in three columns to produce a single 'name' column.  Later, I'll compare that 'name' column to a similar column in the prices dataset and hopefully be able to accurately combine all this information for further analysis!

In [56]:
# Create an aggregate column from the all_data dataframe to match against the prices 'ndc description' dataframe with fuzzywuzzy's Levenshtein Distance generator
all_data['ndc_description_agg'] = all_data['trade_name'] + " " + all_data['strength'] + " " + all_data['route']
all_data.head(2)

,ingredient,dosage_form,route,trade_name,strength,applicant,appl_type_x,appl_no,product_no,te_code,...,patent_expire_date_text,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date,ndc_description_agg
0,BUDESONIDE,"AEROSOL, FOAM",RECTAL,UCERIS,2MG/ACTUATION,VALEANT PHARMS INTL,N,205613,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UCERIS 2MG/ACTUATION RECTAL
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM",TOPICAL,BETAMETHASONE VALERATE,0.12%,PERRIGO UK FINCO,A,78337,1,AB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BETAMETHASONE VALERATE 0.12% TOPICAL


In [28]:
prices['NDC Description'].value_counts().head(10)

METFORMIN HCL 500 MG TABLET      22455
METFORMIN HCL 1,000 MG TABLET    18756
LISINOPRIL 20 MG TABLET          17532
AMLODIPINE BESYLATE 5 MG TAB     17420
MONTELUKAST SOD 10 MG TABLET     17061
METFORMIN HCL 850 MG TABLET      16958
AMLODIPINE BESYLATE 10 MG TAB    16789
LISINOPRIL 10 MG TABLET          16727
LOSARTAN POTASSIUM 50 MG TAB     16238
LISINOPRIL 40 MG TABLET          15932
Name: NDC Description, dtype: int64

## __Pricing Data: Cleaning, cleaning, and more cleaning...__
Inconsistencies in the prices dataset (as shown above) require further cleaning of the names on in the prices dataset.  The following corrections will allow me to produce a higher quality match later in the notebook as I attempt to merge all the datasets.

In [27]:
gc.collect()
# Correct inconsistencies in 'NDC Descriptions' (i.e. CAP = CAPSULE, etc.)
cap_regex = re.compile(r'\sCAP*?\Z | \sCP*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(cap_regex, ' CAPSULE')

chw_regex = re.compile(r'[\sCHW]*?\Z | [\sCHEW]*?\Z')
chw_regex2 = re.compile(r'[\sCHEW]*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace((chw_regex or chw_regex2), ' CHEWABLE')

tab_regex = re.compile(r'\sTAB\Z')
tab_regex2 = re.compile(r'\sTAB\s')
prices['NDC Description'] = prices['NDC Description'].str.replace((tab_regex or tab_regex2), ' TABLET')

syr_regex = re.compile(r'\sSYR*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(syr_regex, ' SYRINGE')

crm_regex = re.compile(r'\sCRM*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(crm_regex, ' CREAM')

sl_regex = re.compile(r'\sSL*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(sl_regex, ' SUB-LINGUAL')

foam_regex = re.compile(r'\sFOAM*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(foam_regex, ' FOAM')

autoinj_regex = re.compile(r'\sAUTO\-INJ*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(autoinj_regex, ' INJECTION')

eff_regex = re.compile(r'\sEFF*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(eff_regex, ' EFFERVESCENT')

soln_regex = re.compile(r'\sSOLN*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(soln_regex, ' SOLUTION')

inh_regex = re.compile(r'\sINH*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(inh_regex, ' INHALATION')

hcl_regex = re.compile(r'\sHCL\s*?\Z')
prices['NDC Description'] = prices['NDC Description'].str.replace(hcl_regex, ' HYDROCHLORIDE')

# Remove the space in between the dosage and dosage units
prices['NDC Description'] = prices['NDC Description'].str.replace(' MG', 'MG')
prices['NDC Description'] = prices['NDC Description'].str.replace(' ML', 'ML')
prices['NDC Description'] = prices['NDC Description'].str.replace(' MCG', 'MCG')
# Remove any white extra white space in the column data
prices['NDC Description'] = prices['NDC Description'].str.replace(' +', ' ')

#Print the new cleaned unique_titles
prices['NDC Description'].value_counts()

METFORMIN HCL 500MG TABLET                22455
METFORMIN HCL 1,000MG TABLET              18756
LISINOPRIL 20MG TABLET                    17532
AMLODIPINE BESYLATE 5MG TABLET            17420
MONTELUKAST SOD 10MG TABLET               17061
METFORMIN HCL 850MG TABLET                16958
AMLODIPINE BESYLATE 10MG TABLET           16789
LISINOPRIL 10MG TABLET                    16727
LOSARTAN POTASSIUM 50MG TABLET            16238
LISINOPRIL 40MG TABLET                    15932
LOSARTAN POTASSIUM 100MG TABLET           15730
LISINOPRIL 5MG TABLET                     15105
GABAPENTIN 300MG CAPSULE                  14016
DONEPEZIL HCL 5MG TABLET                  13696
IBUPROFEN 200MG TABLET                    13575
AMLODIPINE BESYLATE 2.5MG TABLET          13157
FINASTERIDE 5MG TABLET                    12918
DONEPEZIL HCL 10MG TABLET                 12793
CITALOPRAM HBR 40MG TABLET                12645
CLOPIDOGREL 75MG TABLET                   12539
OMEPRAZOLE DR 20MG CAPSULE              

In [49]:
price_cols = list(prices.columns.values)
price_cols

['ndc_description',
 'ndc',
 'cost_per_unit_usd',
 'effective_date',
 'pricing_unit',
 'over_the_counter',
 'classification_for_rate_setting',
 'corresponding_generic_drug_nadac_per_unit',
 'corresponding_generic_drug_effective_date',
 'as_of_date']

In [29]:
# Rename/format or drop some columns
prices.rename(columns = {'NDC Description': 'ndc_description', 'NADAC_Per_Unit':'cost_per_unit_usd', 'OTC':'over_the_counter', 'As of Date':'as_of_date'}, inplace=True)
prices.drop(['Pharmacy_Type_Indicator', 'Explanation_Code'], axis=1, inplace=True)

prices.columns = map(str.lower, prices.columns)
gc.collect()
prices.head()

,ndc_description,ndc,cost_per_unit_usd,effective_date,pricing_unit,over_the_counter,classification_for_rate_setting,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date,as_of_date
0,IBUPROFEN 200MG TABLET,70000017504,0.02991,11/21/2018,EA,Y,G,NaN,NaN,12/12/2018
1,OXYCODONE HCL ER 80MG TABLET,115156201,9.02989,11/21/2018,EA,N,G,NaN,NaN,12/12/2018
2,OXYCODONE HCL 15MG TABLET,603499128,0.15188,11/21/2018,EA,N,G,NaN,NaN,12/12/2018
3,KETOCONAZOLE 200MG TABLET,51672402601,1.68457,11/21/2018,EA,N,G,NaN,NaN,12/12/2018
4,NICOTINE 7MG/24HR PATCH,70000011302,1.72464,11/21/2018,EA,Y,G,NaN,NaN,12/12/2018


In [32]:
# Convert Effective_Date column to datetime format
prices['effective_date'] = pd.to_datetime(prices['effective_date'], format='%m/%d/%Y')
prices['as_of_date'] = pd.to_datetime(prices['as_of_date'], format='%m/%d/%Y')

In [52]:
# Drop all prices older than 5 years in the prices dataset
prices_filtered = prices[(prices['effective_date'].dt.year > 2018)]
prices_filtered.describe()

In [54]:
prices_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634922 entries, 50009 to 820642
Data columns (total 10 columns):
ndc_description                              634922 non-null object
ndc                                          634922 non-null int64
cost_per_unit_usd                            634922 non-null float64
effective_date                               634922 non-null datetime64[ns]
pricing_unit                                 634922 non-null object
over_the_counter                             634922 non-null object
classification_for_rate_setting              634922 non-null object
corresponding_generic_drug_nadac_per_unit    19758 non-null float64
corresponding_generic_drug_effective_date    19758 non-null object
as_of_date                                   634922 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 53.3+ MB


In [55]:
# Check the spread of the 'effective_date' column data
prices['effective_date'].dt.year.value_counts()

2018    1318309
2017    1273904
2016    1257914
2015    1222646
2014    1199249
2019     634922
2013     214199
Name: effective_date, dtype: int64

In [56]:
# Check the spread of the 'as_of_date' column data
prices['as_of_date'].dt.year.value_counts()

2018    1296863
2017    1281151
2016    1250949
2015    1222702
2014    1213307
2019     745780
2013     110391
Name: as_of_date, dtype: int64

In [57]:
# prices.set_index('ndc_description')
# all_data.set_index('ndc_description_agg')

In [58]:
prices_filtered.describe()

,ndc,cost_per_unit_usd,corresponding_generic_drug_nadac_per_unit
count,6.349220e+05,634922.000000,19758.000000
mean,3.737704e+10,10.049851,4.712026
std,2.773782e+10,244.629325,20.998328
min,2.143380e+06,0.001810,0.002760
25%,7.815204e+08,0.072520,0.125710
50%,4.948301e+10,0.207090,0.410460
75%,6.275604e+10,0.812930,3.164960
max,7.643904e+10,21331.276860,323.373750


In [59]:
prices_filtered.sort_values(by = 'ndc_description', ascending = True)
all_data.sort_values(by = 'ndc_description_agg', ascending = True)

,ingredient,dosage_form,route,trade_name,strength,applicant,appl_type_x,appl_no,product_no,te_code,...,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date,df;route,ndc_description_agg
30150,ASPIRIN,"TABLET, EXTENDED RELEASE",ORAL,8-HOUR BAYER,650MG,BAYER,N,16030,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"TABLET, EXTENDED RELEASE;ORAL",8-HOUR BAYER 650MG ORAL
2921,METHOXSALEN,CAPSULE,ORAL,8-MOP,10MG,VALEANT PHARM INTL,N,9048,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAPSULE;ORAL,8-MOP 10MG ORAL
13858,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 100MG BASE/VIAL,HOSPIRA,A,40666,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 100MG BASE/VIAL INJECTION
13854,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 100MG BASE/VIAL,ABBOTT,A,89577,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 100MG BASE/VIAL INJECTION
13859,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 100MG BASE/VIAL,HOSPIRA,A,85929,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 100MG BASE/VIAL INJECTION
13853,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 100MG BASE/VIAL,ABBOTT,A,85928,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 100MG BASE/VIAL INJECTION
13857,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 1GM BASE/VIAL,HOSPIRA,A,85932,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 1GM BASE/VIAL INJECTION
13852,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 1GM BASE/VIAL,ABBOTT,A,89580,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 1GM BASE/VIAL INJECTION
13855,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 250MG BASE/VIAL,ABBOTT,A,89578,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 250MG BASE/VIAL INJECTION
13860,HYDROCORTISONE SODIUM SUCCINATE,INJECTABLE,INJECTION,A-HYDROCORT,EQ 250MG BASE/VIAL,HOSPIRA,A,85930,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INJECTABLE;INJECTION,A-HYDROCORT EQ 250MG BASE/VIAL INJECTION


In [60]:
prices_filtered.head()

,ndc_description,ndc,cost_per_unit_usd,effective_date,pricing_unit,over_the_counter,classification_for_rate_setting,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date,as_of_date
50009,BRILINTA 90MG TABLET,186077739,5.98540,2019-01-01,EA,N,B,NaN,NaN,2018-12-26
50079,MOVANTIK 25MG TABLET,310197030,11.37879,2019-01-01,EA,N,B,NaN,NaN,2018-12-26
50533,DAYTRANA 30MG/9 HOUR PATCH,68968555503,11.54818,2019-01-01,EA,N,B,NaN,NaN,2018-12-26
50692,BYDUREON 2MG PEN INJECT,310653004,167.93958,2019-01-01,EA,N,B,NaN,NaN,2018-12-26
51177,"KOMBIGLYZE XR 5-1,000MG TABLET",310614530,13.45044,2019-01-01,EA,N,B,NaN,NaN,2018-12-26


In [61]:
prices_filtered = prices_filtered.astype(object)
all_data = all_data.astype(object)

In [ ]:
# Drop all unnecessary columns

new_prices = prices.drop(['ndc', 'cost_per_unit_usd', 'effective_date',
                          'pricing_unit', 'over_the_counter', 'classification_for_rate_setting',
                          'corresponding_generic_drug_nadac_per_unit',
                          'corresponding_generic_drug_effective_date', 'as_of_date'], axis = 1)

In [26]:
# In preparation for merging, create a new dataset that contains the bare minimum data needed for the fuzzy string comparison
new_all_data = all_data.drop(['ingredient', 'dosage_form', 'route', 'trade_name', 'strength',
                              'applicant', 'appl_type_x', 'appl_no', 'product_no', 'te_code',
                              'approval_date', 'rld', 'rs', 'type', 'applicant_full_name',
                              'appl_type_y', 'patent_no', 'patent_expire_date_text',
                              'drug_substance_flag', 'drug_product_flag', 'patent_use_code',
                              'delist_flag', 'submission_date', 'appl_type', 'exclusivity_code',
                              'exclusivity_date'], axis = 1)


In [63]:
new_all_data.describe()

,ndc_description_agg
count,58500
unique,14488
top,IMBRUVICA 140MG ORAL
freq,822


In [47]:
# Option 2.0

# Define matching function that will be used to provide a comparison of strings (drug names, strengths, and routes) for later merging of datasets
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.token_set_ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

## __Name matching__
I've found that very little of the data in the drug pricing dataset and the patent dataset overlaps.  This is good and bad.  Good, because it gives me more data to play with.  Bad because it'll be more difficult to match up the data in each set.

I've found that there's a python package called 'fuzzywuzzy' which produces a Levenshtein score (effectively a way to compare the similarity of two strings).  I plan to use the score as I compare the ndc_description (read: drug name) from one dataset to an aggregate of three columns in the other dataset (trade_name, strength, route) that should produce a similar drug name.

Because I had a lot of problems with the processing of these fuzzy strings, I had to break them up into batches so that I'd have more control over the process (than a loop would give me).

In [ ]:
# Option 2.1 - works (w/o Dask!)
# Runs the function above
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices['ndc_description'][:1000]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_all_data['ndc_description_agg'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table1 = pd.DataFrame(dict_list)
# Display results
merge_table1

In [49]:
# Option 2.1 - works (w/o Dask!)
# Runs the function above
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices['ndc_description'][1001:2000]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_all_data['ndc_description_agg'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table2 = pd.DataFrame(dict_list)
# Display results
# merge_table2

In [50]:
# Option 2.1 - works (w/o Dask!)
# Runs the function above
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices['ndc_description'][2001:3000]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_all_data['ndc_description_agg'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table3 = pd.DataFrame(dict_list)
# Display results
# merge_table3

In [51]:
# Option 2.1 - works (w/o Dask!)
# Runs the function above
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices['ndc_description'][3001:4000]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_all_data['ndc_description_agg'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table4 = pd.DataFrame(dict_list)
# Display results
# merge_table4

In [52]:
# Option 2.1 - works (w/o Dask!)
# Runs the function above
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices['ndc_description'][4001:5000]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_all_data['ndc_description_agg'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table5 = pd.DataFrame(dict_list)
# Display results
# merge_table5

In [2]:
# Concatenate all fuzzy merged files (if you turn this on, turn the code in the next cell down off)
# frames = [merge_table1, merge_table2, merge_table3, merge_table4, merge_table5]
# all_merged = pd.concat(frames)

In [ ]:
#Bring in all tables instead of run the loops to generate fuzz scores again (turn this off if you want to run the cell immediately above)
merge_table1 = pd.read_csv('merge_table1')
merge_table2 = pd.read_csv('merge_table2')
merge_table3 = pd.read_csv('merge_table3')
merge_table4 = pd.read_csv('merge_table4')
merge_table5 = pd.read_csv('merge_table5')
merge_all = pd.concat([merge_table1, merge_table2, merge_table3, merge_table4, merge_table5])

In [64]:
fuzzy_prices = pd.merge(prices, merge_all, on = ['ndc_description'], how = 'inner')
fuzzy_prices.head()

,ndc_description,ndc,cost_per_unit_usd,effective_date,pricing_unit,over_the_counter,classification_for_rate_setting,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date,as_of_date,Unnamed: 0,ndc_description_agg,score
0,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,0,IBUPROFEN 200MG ORAL,86
1,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,115,IBUPROFEN 200MG ORAL,86
2,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,831,IBUPROFEN 200MG ORAL,86
3,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,945,IBUPROFEN 200MG ORAL,86
4,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,906,IBUPROFEN 200MG ORAL,86


In [ ]:
# Clean up a bit to free up some space
del merge_table1
del merge_table2
del merge_table3
del merge_table4
del merge_table5

In [66]:
# Reduce the size of fuzzy_prices by taking out any values that don't have a high match (fuzz) score
fuzzy_prices = fuzzy_prices[fuzzy_prices['score'] >= 85]
fuzzy_prices.head()

,ndc_description,ndc,cost_per_unit_usd,effective_date,pricing_unit,over_the_counter,classification_for_rate_setting,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date,as_of_date,Unnamed: 0,ndc_description_agg,score
0,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,0,IBUPROFEN 200MG ORAL,86
1,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,115,IBUPROFEN 200MG ORAL,86
2,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,831,IBUPROFEN 200MG ORAL,86
3,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,945,IBUPROFEN 200MG ORAL,86
4,IBUPROFEN 200MG TABLET,70000017504,0.02991,2018-11-21,EA,Y,G,NaN,NaN,2018-12-12,906,IBUPROFEN 200MG ORAL,86


In [67]:
# Crashes system due to low memory
all_merged_data = pd.merge(fuzzy_prices, new_all_data, on = ['ndc_description_agg'], how = 'inner')
all_merged_data.head()

KeyboardInterrupt: 

I've learned that it's very helpful to regularly export your data if you're frequently maxing out your machine's capabilities :)

In [53]:
# Export all fuzz files (only need if fuzz is running particularly slow)
merge_table1 = merge_table1.to_csv('merge_table1')  #processed (records :1000)
merge_table2 = merge_table2.to_csv('merge_table2')  #processed (records 1001:2000)
merge_table3 = merge_table3.to_csv('merge_table3')  #processed (records 2001:3000)
merge_table4 = merge_table4.to_csv('merge_table4')  #processed (records 3001:4000)
merge_table5 = merge_table5.to_csv('merge_table5')  #processed (records 3001:4000)

In [28]:
# Export all merged files (if you could process them all together)
fuzzy_prices = fuzzy_prices.to_csv('fuzzy_prices')
all_data = all_data.to_csv('all_data.csv')

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [ ]:
# Export data from all files above as single file (if you could process them all together)
all_merged_data = all_merged_data.to_csv('all_merged_data')  #prices, patents, products, exclusivity files

## __There's work still to do!__

Plenty of questions that this dataset can still answer.  Here are a few of the items on those lists:

To do: 
* Setup data downloads from APIs
* Introduce data via Dask from the very beginning (hopefully will improve merge times)
* Specify column for 'days until patent expiry'

Questions to answer (I'll expand on these as I continue to explore the dataset):
* Correlation between variables (particularly drug prices and patent dates)
* Generic vs. Brand drug prices
* Price by active ingredient (correlation & sorting)
* 

<p>
    <p>
        <p>




### __Everything Beyond this point is an effort to quicken the above processes with Dask (parallel processing)__
 
             
             


In [ ]:
import dask.dataframe as dd
from dask.distributed import Client
client = Client()

client

In [ ]:
prices_ddf = dd.from_pandas(prices, npartitions=1)
all_data_ddf = dd.from_pandas(all_data, npartitions=1)

In [ ]:
prices_filtered_ddf = dd.from_pandas(prices_filtered, chunksize = 25e6) #prices_filtered: 404.2MB
all_data_ddf = dd.from_pandas(all_data, chunksize = 25e6) #all_data: 88.7MB

In [ ]:
# Option 1.0
def fuzzy_score(str1, str2):
    return fuzz.token_set_ratio(str1, str2)

def helper(orig_string, slave_df): # add Client in here?
    slave_df['score'] = slave_df['ndc_description_agg'].apply(lambda x: fuzzy_score(x,orig_string))
    #return my_value corresponding to the highest score
    return slave_df.loc[slave_df.ndc_description_agg.idxmax(),'ndc_description']

dmaster = dd.from_pandas(all_data, npartitions=8) # add Client in here?
dmaster['ndc_description'] = dmaster.ndc_description.apply(lambda x: helper(prices_filtered_ddf, prices_filtered_ddf, meta=(x, 'f8'))

In [ ]:
# Option 1.1
# dmaster.computer(schedule = 'processes')  #original line of code
final = dmaster.scatter  #try this instead


In [ ]:
# Option 2.0 (dask starts below)

# Define matching function
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.token_set_ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [ ]:
# Option 2.1 - (trying w/ Dask!)
# Runs the function above
gc.collect()
new_prices_filtered_ddf = dd.from_pandas(new_prices, chunksize = int(25e6) #prices_filtered: 404.2MB
new_all_data_ddf = dd.from_pandas(new_all_data, chunksize = int(25e6) #all_data: 88.7MB

# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in new_prices_filtered_ddf['ndc_description_agg'][:100]:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, new_prices_filtered_ddf['ndc_description'], 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc_description_agg' : name})
    dict_.update({'ndc_description' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

In [ ]:
# Option 3.1
# Merge two dataframes (new_prices, new_all_data) and call .applymap() - applies to the entire with a lambda calling the fuzzy_score function defined above
# Merge
agg_names = new_prices + new_all_data

# Call 
agg_names['score'] = agg_names['ndc_description_agg'].applymap(lambda x: x.fuzz(x, ndc_description),ndc_description_agg)
                                                          


In [ ]:
# Option 4 - my approach
test_data = []
for each in prices_filtered['ndc_description'][100]:
    a = 1
    b = 1
    while a < 101: #len(all_data['ndc_description_agg'])
        testing = all_data['ndc_description_agg'][a]
        rating = fuzz.ratio(testing, each) # Compare the two strings and save the result
        # print(rating, end='\r')
        if rating >= 80:
            test_data.append([each, all_data['ndc_description_agg'], rating])
            #prices_filtered.append(each, inplace = True)
        a += 1
    b += 1
